In [14]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install pymatreader

In [16]:
import pandas as pd
import numpy as np

In [17]:
def kinz_data(data):
  numbodies_list = data["kinzData"]['numbodies']
  req_index = []

  for i in range(len(numbodies_list)):
    if numbodies_list[i]==1:
      req_index.append(i)


  kinz_arr = np.empty([0,97])

  for i in req_index:

    timeStamps = np.array(data["kinzData"]['timeStamps'][i])
    kinz_x = data["kinzData"]['bodies'][i]['Position3d'][0]
    kinz_y = data["kinzData"]['bodies'][i]['Position3d'][1]
    kinz_z = data["kinzData"]['bodies'][i]['Position3d'][2]

    d = np.concatenate((timeStamps,data["kinzData"]['bodies'][i]['Position3d'][0], data["kinzData"]['bodies'][i]['Position3d'][1],
                        data["kinzData"]['bodies'][i]['Position3d'][2]), axis=None)


    kinz_arr = np.append(kinz_arr, [d], axis=0)



  arr_walking_speed = np.array([data["walking_speed"]]*len(req_index))
  arr_trial_num = np.array([data["trial_num"]]*len(req_index))
  arr_patientID = np.array([data["patientID"]]*len(req_index))

  #res_kinz_arr = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,kinz_arr))






def acc_data(data):

  arr_walking_speed = np.array([data["walking_speed"]]*len(data["accelData"]['timeStamps']))
  arr_stopwatch = np.array([data["stopwatch"]]*len(data["accelData"]['timeStamps']))
  arr_trial_num = np.array([data["trial_num"]]*len(data["accelData"]['timeStamps']))
  arr_patientID = np.array([data["patientID"]]*len(data["accelData"]['timeStamps']))

  #res = np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))
  return np.column_stack((arr_patientID,arr_trial_num,arr_walking_speed,arr_stopwatch,data["accelData"]['timeStamps'],data["accelData"]['timeSeries']))





import numpy as np

def kinz_time(kinz_data):
    frames_num = len(kinz_data)
    time_in_sec = np.zeros(frames_num)
    time0 = kinz_data[0]  # time zero for the acquisition

    for i in range(frames_num):
        time_dbl = kinz_data[i] - time0  # relative time (double precision)
        time_in_sec[i] = time_dbl / 1e9  # Convert time to seconds.

    mean_time_diff = np.mean(np.diff(time_in_sec))  # Mean time difference
    fs = 1 / mean_time_diff  # Sampling rate

    return time_in_sec, fs

In [7]:
from pymatreader import read_mat
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm

#data_path = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/Cleaned Data'

data_path = "/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024"

In [ ]:
# for all mat file

"""all_file_path=[]

for i in sorted(glob(data_path+"/*")):
  all_file_path = all_file_path+glob(i+"/*.mat")
  print(i)"""

In [10]:
#for clean preproccessed data
import os
all_file_path=[]

for i in sorted(glob(data_path+"/*")):
    for j in glob(i+"/*"):
        if 'TS' in os.path.basename(j).split("-"):
            all_file_path.append(j)

In [11]:
all_file_path

['/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial1_Regular_20220307_111113.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial5_Fast_20220307_111347.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial2_Regular_20220307_111139.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial4_Fast_20220307_111234.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial3_Fast_20220307_111208.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial6_Fast_20220307_111427.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/001-LO/TS-Cln-001-LO_Trial6_Fast_20220307_111722.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/002-RJ/TS-Cln-002-RJ_Trial4_Fast_20220311_100659.mat',
 '/content/drive/MyDrive/Iqram Sir/OneDrive_1_6-12-2024/002-RJ/TS-Cln-002-RJ_Trial4_Fast_20220311_100541.m

In [12]:
read_mat(all_file_path[0])

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Tue Jun  4 12:04:19 2024',
 '__version__': '1.0',
 '__globals__': [],
 'accel_energy': array([[2.01999255e-05, 8.89671405e-06, 1.98536528e-07, 4.80889868e-08,
         3.43106550e-06, 8.92035676e-06],
        [4.21610353e-06, 9.45410661e-06, 5.86701262e-07, 1.19682109e-06,
         8.14110011e-06, 1.96705733e-06],
        [7.42636032e-07, 9.19725965e-07, 6.35215593e-07, 1.15043391e-06,
         6.81433970e-07, 5.55391863e-07],
        ...,
        [5.70672229e-08, 4.38350770e-08, 6.68218023e-08, 3.09131551e-08,
         1.28920793e-07, 3.17894137e-08],
        [8.19358732e-08, 5.10268593e-08, 3.97833515e-08, 1.14223045e-08,
         4.37020002e-08, 5.77700083e-08],
        [2.01129426e-08, 7.55306056e-08, 3.15665263e-08, 5.44397200e-08,
         1.86808358e-07, 8.23734548e-08]]),
 'accel_energy_total': array([3.10174637e+02, 1.44858216e+02, 2.43550993e+01, 8.84180908e+00,
        6.70987649e+00, 2.85818700e+00, 3.82810

TS-Cln- Data Files

These files are composed of time series vectors or matrices

•	t_uniform: time vector for both Kinect and accel time series<br><br>
•	accel_energy: root mean square signal energy for each of the 6 sensor locations, obtained using a moving window equivalent to the sampling interval. Column index corresponds to the sensor index from the instrumentation plan<br> <br>
•	accel_energy_total: maximum signal energy across all 6 sensors, after each has been normalized by the standard deviation of its energy signal<br><br>
•	footfall_event_times: time instances from t_uniform where a significant peak was detected in accel_energy_total. <br><br>
•	raw_accel: raw acceleration data from the 6 sensor locations. Column index corresponds to the sensor index from the instrumentation plan.<br><br>
•	t_accel: time vector for raw_accel, sampling rate is different than that of t_uniform<br><br>
•	joint_positions: Kinect body tracking joint positions over time. First dimension is time, second dimension is the XYZ coordinate, and third dimension is the joint index according to the Kinect convention (minus one).<br><br>
•	Pelvis_pos: the total distance travelled by the pelvis in 3D space, derived from joint_positions.<br><br>
•	Right_foot_swing: right ankle Z position-pelvis Z position.<br><br>
•	left_foot_swing: left ankle Z position-pelvis Z position.

In [ ]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

#file_save_destination = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Code/Output Data/Kinect Data/kinzData_panda_'+formatted_datetime+'.csv'

file_save_destination = "task1.csv"


if os.path.exists(file_save_destination)==False:
  pd.DataFrame(np.empty([0,100]), columns=['patientID','trial_num','walking_speed','timeStamps',
  'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
  'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
  'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                ]).to_csv(file_save_destination,index=False)



try:

  for path in tqdm(all_file_path):
    data = read_mat(path)
    temp_df = pd.DataFrame(kinz_data(data), columns=['patientID','trial_num','walking_speed','timeStamps',
    'PELVIS_X', 'SPINE_NAVAL_X', 'SPINE_CHEST_X', 'NECK_X', 'CLAVICLE_LEFT_X', 'SHOULDER_LEFT_X', 'ELBOW_LEFT_X', 'WRIST_LEFT_X', 'HAND_LEFT_X', 'HANDTIP_LEFT_X', 'THUMB_LEFT_X', 'CLAVICLE_RIGHT_X', 'SHOULDER_RIGHT_X', 'ELBOW_RIGHT_X', 'WRIST_RIGHT_X', 'HAND_RIGHT_X', 'HANDTIP_RIGHT_X', 'THUMB_RIGHT_X', 'HIP_LEFT_X', 'KNEE_LEFT_X', 'ANKLE_LEFT_X', 'FOOT_LEFT_X', 'HIP_RIGHT_X', 'KNEE_RIGHT_X', 'ANKLE_RIGHT_X', 'FOOT_RIGHT_X', 'HEAD_X', 'NOSE_X', 'EYE_LEFT_X', 'EAR_LEFT_X', 'EYE_RIGHT_X', 'EAR_RIGHT_X',
    'PELVIS_Y', 'SPINE_NAVAL_Y', 'SPINE_CHEST_Y', 'NECK_Y', 'CLAVICLE_LEFT_Y', 'SHOULDER_LEFT_Y', 'ELBOW_LEFT_Y', 'WRIST_LEFT_Y', 'HAND_LEFT_Y', 'HANDTIP_LEFT_Y', 'THUMB_LEFT_Y', 'CLAVICLE_RIGHT_Y', 'SHOULDER_RIGHT_Y', 'ELBOW_RIGHT_Y', 'WRIST_RIGHT_Y', 'HAND_RIGHT_Y', 'HANDTIP_RIGHT_Y', 'THUMB_RIGHT_Y', 'HIP_LEFT_Y', 'KNEE_LEFT_Y', 'ANKLE_LEFT_Y', 'FOOT_LEFT_Y', 'HIP_RIGHT_Y', 'KNEE_RIGHT_Y', 'ANKLE_RIGHT_Y', 'FOOT_RIGHT_Y', 'HEAD_Y', 'NOSE_Y', 'EYE_LEFT_Y', 'EAR_LEFT_Y', 'EYE_RIGHT_Y', 'EAR_RIGHT_Y',
    'PELVIS_Z', 'SPINE_NAVAL_Z', 'SPINE_CHEST_Z', 'NECK_Z', 'CLAVICLE_LEFT_Z', 'SHOULDER_LEFT_Z', 'ELBOW_LEFT_Z', 'WRIST_LEFT_Z', 'HAND_LEFT_Z', 'HANDTIP_LEFT_Z', 'THUMB_LEFT_Z', 'CLAVICLE_RIGHT_Z', 'SHOULDER_RIGHT_Z', 'ELBOW_RIGHT_Z', 'WRIST_RIGHT_Z', 'HAND_RIGHT_Z', 'HANDTIP_RIGHT_Z', 'THUMB_RIGHT_Z', 'HIP_LEFT_Z', 'KNEE_LEFT_Z', 'ANKLE_LEFT_Z', 'FOOT_LEFT_Z', 'HIP_RIGHT_Z', 'KNEE_RIGHT_Z', 'ANKLE_RIGHT_Z', 'FOOT_RIGHT_Z', 'HEAD_Z', 'NOSE_Z', 'EYE_LEFT_Z', 'EAR_LEFT_Z', 'EYE_RIGHT_Z', 'EAR_RIGHT_Z'
                                                  ])

    columns_to_convert = temp_df.columns.difference(['patientID','trial_num', 'walking_speed'])
    temp_df[columns_to_convert] = temp_df[columns_to_convert].astype(float)

    k = kinz_time(temp_df["timeStamps"])
    temp_df["timeStamps"] = k[0]
    temp_df.to_csv(file_save_destination, mode='a', index=False, header=False)
    break

except:
  print("The Subject folder or main path unusual file or folder below.....Please remove the file or the folder.....")
  print(path)

  0%|          | 0/309 [00:00<?, ?it/s]

/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


In [ ]:
temp_df

,patientID,trial_num,walking_speed,timeStamps,PELVIS_X,SPINE_NAVAL_X,SPINE_CHEST_X,NECK_X,CLAVICLE_LEFT_X,SHOULDER_LEFT_X,...,HIP_RIGHT_Z,KNEE_RIGHT_Z,ANKLE_RIGHT_Z,FOOT_RIGHT_Z,HEAD_Z,NOSE_Z,EYE_LEFT_Z,EAR_LEFT_Z,EYE_RIGHT_Z,EAR_RIGHT_Z
0,001-LO,1,Regular,0.000000,509.953796,511.707184,519.876770,542.541321,513.445496,405.303040,...,913.327148,935.423889,861.115906,933.341858,896.654175,960.068970,921.099670,839.498840,952.534424,930.024780
1,001-LO,1,Regular,0.031953,516.699402,520.979065,537.396118,549.385742,519.529419,405.027405,...,927.616211,922.977234,785.082581,848.691956,957.863037,947.431702,958.439331,941.279785,986.675720,1040.982178
2,001-LO,1,Regular,0.064029,535.292725,549.477844,558.815186,554.463379,526.307251,409.880524,...,1001.933350,942.371033,774.065186,833.417175,999.197083,998.734680,1010.169617,986.934570,1038.330200,1086.423584
3,001-LO,1,Regular,0.239935,528.215088,525.848572,523.920715,513.119446,486.163452,370.992065,...,1297.838989,1347.285522,1240.194946,1310.312988,1266.138428,1403.958984,1386.834961,1294.465698,1387.365356,1288.052490
4,001-LO,1,Regular,0.271985,525.315613,525.240662,525.198303,519.277588,491.326874,373.072540,...,1345.556396,1420.827881,1293.354126,1359.483521,1314.222046,1449.538452,1428.695557,1326.911743,1439.558350,1354.007446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,001-LO,1,Regular,4.671977,305.414069,283.564899,266.337668,248.141546,216.611869,77.172778,...,7683.455537,7785.870712,7656.073791,7749.564852,7611.999388,7754.598487,7713.899917,7611.184767,7714.755802,7596.810072
95,001-LO,1,Regular,4.736101,275.946655,274.226379,272.905426,272.781342,238.690140,96.460548,...,7748.098633,7879.586914,7929.703613,8060.992188,7702.207520,7861.719238,7830.703125,7721.208984,7830.533203,7706.862305
96,001-LO,1,Regular,4.767949,265.641327,257.973358,252.833878,259.411194,224.871277,96.088661,...,7791.557617,7843.167480,7862.297363,8001.824219,7742.589355,7896.048828,7866.208984,7764.863281,7861.868164,7736.953125
97,001-LO,1,Regular,4.832074,256.468170,243.115814,232.112411,230.675339,195.589493,60.753048,...,7899.273438,7911.373535,7889.722656,8025.136719,7838.875000,7932.834961,7888.795410,7824.206055,7880.759766,7779.110840


In [ ]:
import os
from datetime import datetime

current_datetime = datetime.now()
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M%p")

file_save_destination_1 = '/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Code/Output Data/ACC Data/ACC_Data_panda_'+formatted_datetime+'.csv'

#file_save_destination_1 = "task2.csv"

if os.path.exists(file_save_destination_1)==False:
  pd.DataFrame(np.empty([0,12]), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3','acc_4', 'acc_5', 'acc_6','acc_7']).to_csv(file_save_destination_1,index=False)




for path in tqdm(all_file_path):
  data = read_mat(path)
  temp_df = pd.DataFrame(acc_data(data), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3',
                                  'acc_4', 'acc_5', 'acc_6','acc_7'])


  columns_to_convert = temp_df.columns.difference(['patientID','trial_num','walking_speed','stopwatch'])
  temp_df[columns_to_convert] = temp_df[columns_to_convert].astype(float)

  #k = kinz_time(temp_df["timeStamps"])
  #temp_df["timeStamps"] = k[0]

  #temp_df.to_csv(file_save_destination_1, mode='a', index=False, header=False)
  break



  0%|          | 0/309 [00:00<?, ?it/s]

/home/iqh4001/.conda/envs/jupyter-notebook/lib/python3.10/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


In [ ]:
temp_df

,patientID,trial_num,walking_speed,stopwatch,timeStamps,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7
0,001-LO,1,Regular,5.2234304,-5.200096,0.000140,-0.000068,-0.000604,-0.000376,-0.000250,-0.000145,0.000196
1,001-LO,1,Regular,5.2234304,-5.200037,0.000134,-0.000074,-0.000599,-0.000376,-0.000241,-0.000146,0.000193
2,001-LO,1,Regular,5.2234304,-5.199979,0.000129,-0.000080,-0.000593,-0.000375,-0.000233,-0.000146,0.000190
3,001-LO,1,Regular,5.2234304,-5.199920,0.000123,-0.000085,-0.000587,-0.000376,-0.000224,-0.000147,0.000186
4,001-LO,1,Regular,5.2234304,-5.199862,0.000118,-0.000090,-0.000580,-0.000376,-0.000216,-0.000147,0.000182
...,...,...,...,...,...,...,...,...,...,...,...,...
284908,001-LO,1,Regular,5.2234304,11.493732,-0.000668,-0.000213,0.000293,-0.000694,-0.000112,0.000265,0.000544
284909,001-LO,1,Regular,5.2234304,11.493791,-0.000668,-0.000212,0.000293,-0.000693,-0.000113,0.000265,0.000543
284910,001-LO,1,Regular,5.2234304,11.493849,-0.000668,-0.000210,0.000292,-0.000691,-0.000114,0.000264,0.000542
284911,001-LO,1,Regular,5.2234304,11.493908,-0.000667,-0.000209,0.000292,-0.000690,-0.000115,0.000264,0.000542


In [ ]:
#path='/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/Cleaned Data/001-LO/001-LO_Trial1_Regular_20220307_111113.mat'
path='/home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/AI_Frailty-MATLAB/001-LO/Cln-001-LO_Trial1_Regular_20220307_111113.mat'

data = read_mat(path)
temp_df = pd.DataFrame(acc_data(data), columns=['patientID','trial_num','walking_speed','stopwatch','timeStamps','acc_1', 'acc_2', 'acc_3',
                                'acc_4', 'acc_5', 'acc_6','acc_7'])


columns_to_convert = temp_df.columns.difference(['patientID','trial_num','walking_speed','stopwatch'])
temp_df[columns_to_convert] = temp_df[columns_to_convert].astype(float)


OSError: The file /home/iqh4001/Iqram_WCM/MADE Lab/Kinect Project/Dataset/Data/AI_Frailty-MATLAB/001-LO/Cln-001-LO_Trial1_Regular_20220307_111113.mat does not exist.

In [ ]:
temp_df

,patientID,trial_num,walking_speed,stopwatch,timeStamps,acc_1,acc_2,acc_3,acc_4,acc_5,acc_6,acc_7
0,001-LO,1,Regular,5.2234304,0.000000,0.000579,0.000008,0.000856,0.000474,0.000279,0.000359,0.000340
1,001-LO,1,Regular,5.2234304,0.000059,0.000710,-0.000010,0.000816,0.000491,0.000290,0.000349,0.000315
2,001-LO,1,Regular,5.2234304,0.000117,0.000692,-0.000034,0.000854,0.000545,0.000281,0.000420,0.000325
3,001-LO,1,Regular,5.2234304,0.000176,0.000658,-0.000094,0.000843,0.000471,0.000119,0.000447,0.000325
4,001-LO,1,Regular,5.2234304,0.000234,0.000666,-0.000130,0.000765,0.000395,0.000122,0.000411,0.000400
...,...,...,...,...,...,...,...,...,...,...,...,...
294907,001-LO,1,Regular,5.2234304,17.279707,0.000480,0.000142,0.000281,0.000898,0.000485,0.000581,0.000941
294908,001-LO,1,Regular,5.2234304,17.279766,0.000757,0.000066,0.000199,0.001132,0.000596,0.000538,0.000843
294909,001-LO,1,Regular,5.2234304,17.279824,0.000947,0.000028,0.000142,0.001295,0.000830,0.000499,0.000705
294910,001-LO,1,Regular,5.2234304,17.279883,0.000952,0.000117,0.000155,0.001238,0.000908,0.000336,0.000568
